In [1]:
import numpy as np
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import requests
from scipy.optimize import minimize
from scipy.sparse import hstack

In [9]:
def download_data(url):
# data_url = 'https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/regression/housing_scale'
    data = requests.get(url)
    file_name = url.split('/')[-1] + '.svm'
    with open(file_name, 'w') as f:
        f.write(data.text)
    return file_name

In [10]:
a9a = download_data('https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/a9a')
a9at = download_data('https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/a9a.t')

In [3]:
X, y = load_svmlight_file('a9a.svm')
X_test, y_test = load_svmlight_file('a9a.t.svm')

In [4]:
def sigmoid(z):  
    return 1 / (1 + np.exp(-z))

In [26]:
def loss_fn(theta, X, y, learning_rate):  
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    print(X)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (learning_rate / 2 * len(X)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return np.sum(first - second) / (len(X)) + reg

In [6]:
def gradient_with_loop(theta, X, y, learning_rate):  
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)

    error = sigmoid(X * theta.T) - y

    for i in range(parameters):
        term = np.multiply(error, X[:,i])

        if (i == 0):
            grad[i] = np.sum(term) / len(X)
        else:
            grad[i] = (np.sum(term) / len(X)) + ((learning_rate / len(X)) * theta[:,i])

    return grad

In [7]:
def gradient(theta, X, y, learning_rate):  
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    parameters = int(theta.ravel().shape[1])
    error = sigmoid(X * theta.T) - y

    grad = ((X.T * error) / len(X)).T + ((learning_rate / len(X)) * theta)

    # intercept gradient is not regularized
    grad[0, 0] = np.sum(np.multiply(error, X[:,0])) / len(X)

    return np.array(grad).ravel()

In [19]:
def one_vs_all(X, y, num_labels, learning_rate):  
    rows = X.shape[0]
    params = X.shape[1]
    print(rows, params)

    # k X (n + 1) array for the parameters of each of the k classifiers
    all_theta = np.zeros((num_labels, params + 1))

    # insert a column of ones at the beginning for the intercept term
#     X = np.insert(X, 0, values=np.ones(rows), axis=1)
    X = hstack([np.ones((rows, 1)), X])

    # labels are 1-indexed instead of 0-indexed
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))

        # minimize the objective function
        fmin = minimize(fun=loss_fn, x0=theta, args=(X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1,:] = fmin.x

    return all_theta

In [9]:
def predict_all(X, all_theta):  
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]

    # same as before, insert ones to match the shape
#     X = np.insert(X, 0, values=np.ones(rows), axis=1)
    X = np.vstack([np.ones(rows), X])

    # convert to matrices
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)

    # compute the class probability for each class on each training instance
    h = sigmoid(X * all_theta.T)

    # create array of the index with the maximum probability
    h_argmax = np.argmax(h, axis=1)

    # because our array was zero-indexed we need to add one for the true label prediction
    h_argmax = h_argmax + 1

    return h_argmax

In [25]:
all_theta = one_vs_all(X, y, 2, 1)  
all_theta  

32561 123
[[ <32561x124 sparse matrix of type '<class 'numpy.float64'>'
	with 484153 stored elements in COOrdinate format>]]


ValueError: shapes (1,1) and (124,1) not aligned: 1 (dim 1) != 124 (dim 0)